In [257]:
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset

In [258]:
class TitanicDataset(Dataset):
    def __init__(self, path):
      # Sex, Ticket, Cabin, Embarked
      x_headers = ['Pclass','Age','SibSp','Parch', 'Fare']
      y_headers = ['Survived']
      data = pd.read_csv(path)
      x_data = data.loc[:, x_headers].ffill(0)
      y_data = data.loc[:, y_headers]
      self.len = len(x_data)
      self.x_data = torch.from_numpy(x_data.to_numpy()).float()
      self.y_data = torch.from_numpy(y_data.to_numpy()).float()

    def __getitem__(self, index): 
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [259]:
train_dataset = TitanicDataset(os.path.join('../../tmp/titanic', 'train.csv'))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

/var/folders/ms/23y4kqr569g2hgfrfjmp515m0000gn/T/ipykernel_2914/42285370.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.ffill will be keyword-only.
  x_data = data.loc[:, x_headers].ffill(0)


In [260]:
test_data = [
  [[3., 34.5, 0., 0., 7.8292], 0.],
  [[3., 47., 1., 0., 7.], 1.],
  [[2., 62., 0., 0., 9.6875], 0.],
  [[3., 27., 0., 0., 8.6625], 0.],
]


In [261]:
class SigmoidRegressionModel(torch.nn.Module):
  def __init__(self):
    super(SigmoidRegressionModel, self).__init__()
    self.linear1 = torch.nn.Linear(5, 3)
    self.linear2 = torch.nn.Linear(3, 1)
    self.sigmoid = torch.nn.Sigmoid()
  
  def forward(self, x):
    x = self.linear1(x)
    x = self.linear2(x)
    y_pred = self.sigmoid(x)
    return y_pred
    
model = SigmoidRegressionModel()

In [262]:
criterion = torch.nn.BCELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
print('Before training: ', model(torch.tensor(test_data[0][0])))
for epoch in range(1000):
  for i, (x, y) in enumerate(train_loader):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    if(epoch % 10 == 0 and i == 0):
      print(epoch, loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print('After training: ', model(torch.tensor(test_data[0][0])))


Before training:  tensor([0.1364], grad_fn=<SigmoidBackward0>)
0 266.49615478515625
10 1800.0
20 2000.0
30 2100.0
40 2000.0
50 2000.0
60 1900.0
70 1800.0
80 2100.0
90 2400.0
100 1600.0
110 2100.0
120 1800.0
130 2200.0
140 2100.0
150 2200.0
160 1800.0
170 1700.0
180 2100.0
190 1600.0
200 2000.0
210 1600.0
220 1500.0
230 1900.0
240 2000.0
250 2100.0
260 2200.0
270 2100.0
280 2000.0
290 2200.0
300 1800.0
310 2000.0
320 1900.0
330 2100.0
340 2100.0
350 2100.0
360 1300.0
370 2300.0
380 1800.0
390 2100.0
400 2200.0
410 1900.0
420 2700.0
430 1800.0
440 2400.0
450 2100.0
460 1500.0
470 2000.0
480 2600.0
490 2200.0
500 2300.0
510 2000.0
520 2200.0
530 2000.0
540 2300.0
550 2100.0
560 2600.0
570 1200.0
580 2100.0
590 1400.0
600 1900.0
610 1900.0
620 2100.0
630 2200.0
640 2000.0
650 1900.0
660 1900.0
670 2100.0
680 2400.0
690 2000.0
700 1900.0
710 2200.0
720 1900.0
730 1400.0
740 1600.0
750 1700.0
760 1800.0
770 1900.0
780 2100.0
790 2000.0
800 1400.0
810 1900.0
820 1700.0
830 2400.0
840 2100.0
8

In [266]:
model(torch.tensor(test_data[3][0]))

tensor([1.], grad_fn=<SigmoidBackward0>)